#### Démarches

Le code est brut. Je ne l'ai pas commenté ou quoi que ce soit.

Le plus intéressant, c'est le contexte qui m'a permis de parvenir à cela : j'ai d'abord utilisé une library qui m'a permis de tester plusieurs modèle de manière automatique (pratique pour gagner du temps). Cette library est la suivante : https://mljar.com/automl/

Ce n'est pas la seule, il en existe bien d'autre. Il est aussi aisé de créer sa propre lib pour tester des modèles et faire un rapport pour chaque modèles (ça peut être un bon exercice pour progresser sur le code et la connaissance des modèles !).

Je n'ai pas réussi à faire tourner les tests en entier, mais cela m'a suffit pour voir que Catboost (https://catboost.ai/en/docs/) était un bon candidat pour obtenir un score d'average precision score élevé.

J'ai donc effectué une simple baseline : c'est-à-dire que je prends le modèle avec les paramètres par défaut (j'ai juste ajusté la loss pour qu'elle corresponde à la problématique), et j'ai gardé les données telles qu'elles.

Avec un peu de chance, il s'est avéré que cette simple baseline était la bonne. Catboost sur-performe !

Voilà ma démarche. C'est une démarche souvent employé dans le cadre de compétition de data-science. Cependant, on ne s'arrête pas simplement là. Il s'ensuit plusieurs itérations sur la recherche d'hyper-paramètres du modèle et de feature engineering. C'est d'ailleurs souvent cette dernière qui fait la différence dans le top du classement.

In [4]:
# !pip install mljar-supervised
# !pip install catboost

In [8]:
import pandas as pd
from supervised import AutoML

In [9]:
df = pd.read_csv('data/conversion_data_train.csv')

In [10]:
target_variable = 'converted'
X = df.drop(target_variable, axis=1)
Y = df.loc[:, target_variable]

### AutoML predictions

In [21]:
# Initialize AutoML in Explain/Compete Mode
automl = AutoML(mode="Compete")
automl.fit(X, Y)

In [17]:
automl._best_model

### Catboost

In [2]:
numeric_features = ['age', 'total_pages_visited']
categorical_features = ['new_user', 'country', 'source']

In [11]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(loss_function='Logloss', cat_features=categorical_features, verbose=True)
model.fit(X, Y)

Learning rate set to 0.115048
0:	learn: 0.4002871	total: 134ms	remaining: 2m 13s
1:	learn: 0.2443513	total: 212ms	remaining: 1m 45s
2:	learn: 0.1606698	total: 285ms	remaining: 1m 34s
3:	learn: 0.1158906	total: 363ms	remaining: 1m 30s
4:	learn: 0.0908455	total: 519ms	remaining: 1m 43s
5:	learn: 0.0753024	total: 704ms	remaining: 1m 56s
6:	learn: 0.0662839	total: 894ms	remaining: 2m 6s
7:	learn: 0.0603509	total: 1.1s	remaining: 2m 16s
8:	learn: 0.0558187	total: 1.27s	remaining: 2m 19s
9:	learn: 0.0516494	total: 1.42s	remaining: 2m 20s
10:	learn: 0.0486638	total: 1.7s	remaining: 2m 32s
11:	learn: 0.0474340	total: 1.85s	remaining: 2m 32s
12:	learn: 0.0461871	total: 2.02s	remaining: 2m 33s
13:	learn: 0.0452924	total: 2.16s	remaining: 2m 31s
14:	learn: 0.0446920	total: 2.3s	remaining: 2m 31s
15:	learn: 0.0441613	total: 2.42s	remaining: 2m 28s
16:	learn: 0.0437116	total: 2.55s	remaining: 2m 27s
17:	learn: 0.0432421	total: 2.67s	remaining: 2m 25s
18:	learn: 0.0430213	total: 2.81s	remaining: 2m 

### Submissions

In [13]:
# Read data without labels
data_without_labels = pd.read_csv('data/conversion_data_test.csv')
print('Prediction set (without labels) :', data_without_labels.shape)

Prediction set (without labels) : (31620, 5)


In [14]:
data = {
    # 'converted': automl.predict(data_without_labels)
    'converted': model.predict(data_without_labels)
}

Y_predictions = pd.DataFrame(columns=['converted'],data=data)
Y_predictions.to_csv('submissions/conversion_data_test_predictions_Alexon_V4.csv', index=False)